In [1]:
from sklearn.linear_model import LogisticRegression
from ticket_upgrade_prediction.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [2]:
df = Pipeline().df

2022-09-27 15:45:18.123 | INFO     | ticket_upgrade_prediction.pipeline:read_all_files:34 - loaded booking data
2022-09-27 15:45:22.836 | INFO     | ticket_upgrade_prediction.pipeline:read_all_files:36 - loaded ticket data
2022-09-27 15:45:41.489 | INFO     | ticket_upgrade_prediction.pipeline:read_all_files:38 - loaded flight coupon data
2022-09-27 15:45:41.505 | INFO     | ticket_upgrade_prediction.pipeline:merge_files:49 - 2022-09-27 15:45:41.505446 merging datasets
2022-09-27 15:46:57.174 | INFO     | ticket_upgrade_prediction.pipeline:clean_df:289 - 2022-09-27 15:46:57.173539 cleaning df ...
2022-09-27 15:47:31.698 | INFO     | ticket_upgrade_prediction.pipeline:convert_datetime_columns_to_pandas_format:157 - converted datetime columns to pandas format
2022-09-27 15:47:36.038 | INFO     | ticket_upgrade_prediction.pipeline:filter_wrong_ticket_prices:140 - filtered wrong ticket prices
2022-09-27 15:47:39.452 | INFO     | ticket_upgrade_prediction.pipeline:filter_wrong_booking_windo

In [3]:
df.dropna(inplace=True)
X_train, X_test, y_train, y_test = train_test_split(
        df.drop(['UPGRADED_FLAG'], axis=1), df.iloc[:, 3], test_size=0.2, random_state=42
    )
del(df)

In [7]:
log = LogisticRegression(penalty='l1', solver='saga', class_weight='balanced', verbose=2)
log.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 787 seconds


E:\anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 13.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 13.1min finished


LogisticRegression(class_weight='balanced', penalty='l1', solver='saga',
                   verbose=2)

In [8]:
from ticket_upgrade_prediction.evaluator import Evaluator
import mlflow

In [9]:
#mlflow.set_tracking_uri('http://localhost:5000')
#with mlflow.start_run():
model = log
evaluator = Evaluator(model=model, X=X_test, y=y_test)
print(evaluator.get_all_metrics())
#evaluator.get_all_metrics(to_mlflow=True)
#    mlflow.sklearn.log_model(
#        sk_model=model,
#        artifact_path="sklearn-model",
#        registered_model_name='LASSO'
#    )

Metrics(accuracy=0.8548853631217005, roc_auc=0.9251165788280645, precision=0.006200858752021413, recall=0.9575200918484501, f1=0.012321921274151668, pr_auc=0.007472113575237894)


In [10]:
mlflow.set_tracking_uri('http://localhost:5000')
with mlflow.start_run():
    model = log
    evaluator = Evaluator(model=model, X=X_test, y=y_test)
    print(evaluator.get_all_metrics())
    evaluator.get_all_metrics(to_mlflow=True)
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="sklearn-model",
        registered_model_name='LASSO'
    )

Metrics(accuracy=0.8548853631217005, roc_auc=0.9251165788280645, precision=0.006200858752021413, recall=0.9575200918484501, f1=0.012321921274151668, pr_auc=0.007472113575237894)


E:\anaconda\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'LASSO'.
2022/09/27 17:58:35 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: LASSO, version 1
Created version '1' of model 'LASSO'.


In [14]:
import pickle
with open(r"E:\downloads\model.pkl","rb") as file_handle:
    retrieved_data = pickle.load(file_handle)
    print(retrieved_data.coef_)

[[-5.11801623e-02 -2.43729307e-01  4.06677058e-03 -5.87906768e-04
  -1.87477241e-02 -1.85589454e-03 -4.27163992e-03  1.19812103e-02
   1.23521315e-03  4.90653745e-03 -9.57357703e-06 -1.58225786e-01
  -8.31915624e-02 -2.04786508e-01  6.50258877e-04 -2.70488207e-02
   7.04131408e-03  1.53824818e-03 -1.14744594e-01  0.00000000e+00
   6.78543145e-03 -3.35964784e-02 -1.04163559e-06 -3.34814430e-02
   8.43426106e-03 -3.05977128e-03 -3.27243752e-04 -6.99912892e-03
  -1.43193460e-02  3.12368553e-03 -6.99177165e-03 -5.22537293e-04
  -2.74059817e-04 -1.57730440e-03  2.64664395e-04 -7.56835861e-07
  -4.49349874e-04 -2.77899803e-03 -3.77698175e-05 -2.28132100e-02]]
